# 🎓 Build Your Own RAG Chatbot for NWMSU MS-ACS Program

## Complete Step-by-Step Tutorial for Students

This notebook will guide you through building a complete RAG (Retrieval-Augmented Generation) chatbot for Northwest Missouri State University's MS-ACS program. You'll learn how to create a chatbot that provides context to AI models before answering questions.

### 🎯 What You'll Learn:
- **RAG Architecture**: How retrieval and generation work together
- **Context Retrieval**: How to find relevant information before answering
- **Vector Embeddings**: Convert text to numerical representations
- **LangChain Framework**: Modern tools for building LLM applications
- **Interactive Chatbot**: Create a working chatbot interface

### 🚀 Final Goal:
Build a complete RAG chatbot that can answer questions about the NWMSU MS-ACS program by retrieving relevant context from the university's website.


## 📦 Step 1: Setup and Installation

Let's start by installing all the required packages for our RAG chatbot. This will take a few minutes as we download the necessary libraries.


In [ ]:
# Install required packages for our RAG chatbot
print("📦 Installing packages... This may take a few minutes.")

# Core RAG packages
!pip install -q langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.0

# AI/ML packages
!pip install -q sentence-transformers==5.1.1 transformers==4.57.1 torch==2.8.0

# Web scraping and data processing
!pip install -q requests beautifulsoup4 lxml

# Interactive widgets for our chatbot interface
!pip install -q ipywidgets

# Optional: Neo4j for advanced features (we'll use FAISS for simplicity)
!pip install -q faiss-cpu

print("✅ All packages installed successfully!")
print("🎉 Ready to build your RAG chatbot!")


## 📚 Step 2: Import Libraries

Now let's import all the libraries we need to build our RAG chatbot. These libraries will help us with:
- **LangChain**: Framework for building LLM applications
- **Transformers**: AI models for text processing
- **FAISS**: Vector database for similarity search
- **Web scraping**: Get data from NWMSU website


In [ ]:
# Import all the libraries we need for our RAG chatbot
import os
import warnings
import time
from typing import List, Tuple
import re

# LangChain - Our main framework for building the chatbot
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_community.vectorstores import FAISS

# AI/ML libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# Interactive interface for our chatbot
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

print("✅ All libraries imported successfully!")
print("🚀 Ready to start building your RAG chatbot!")


## 🤖 Step 3: Build Your RAG Chatbot Class

Now we'll create the main class for our RAG chatbot. This class will handle:
- **Data Loading**: Get information from NWMSU website
- **Text Processing**: Split documents into manageable chunks
- **Vector Storage**: Create embeddings for similarity search
- **Context Retrieval**: Find relevant information for questions
- **Answer Generation**: Use AI to generate responses


In [ ]:
class NWMSURAGChatbot:
    """RAG Chatbot for NWMSU MS-ACS Program - Built by Students"""
    
    def __init__(self, use_gpu=False):
        print("🚀 Initializing your RAG Chatbot...")
        print("📚 This chatbot will help students learn about NWMSU MS-ACS program")
        
        self.device = "cuda" if use_gpu and torch.cuda.is_available() else "cpu"
        print(f"📱 Using device: {self.device}")
        
        # Initialize embeddings - these convert text to numbers for similarity search
        print("🔤 Loading text embeddings...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': self.device}
        )
        
        # Initialize the AI language model
        print("🧠 Loading AI language model...")
        self.llm = self._setup_llm()
        
        print("✅ Chatbot initialized successfully!")
    
    def _setup_llm(self):
        """Setup the AI language model that will generate answers"""
        print("    📥 Downloading AI model... (this may take a few minutes)")
        model_id = "google/flan-t5-base"  # A good model for question-answering
        
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            low_cpu_mem_usage=True
        )
        
        # Create a pipeline for text generation
        pipe = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            max_length=512,
            device=0 if self.device == "cuda" else -1
        )
        
        return HuggingFacePipeline(pipeline=pipe)
    
    def _setup_neo4j(self):
        """Setup Neo4j connection"""
        print("🗄️ Setting up Neo4j connection...")
        
        # Neo4j connection details (you can modify these)
        NEO4J_URI = "neo4j+s://813403d3.databases.neo4j.io"
        NEO4J_USERNAME = "neo4j"
        NEO4J_PASSWORD = "4EfVPpL8RGgaSXTN1rudzLxMygGnihSAMtblyskNWz8"
        
        try:
            # Test connection
            driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            with driver.session() as session:
                session.run("RETURN 1")
            
            # Create LangChain Neo4j graph
            graph = Neo4jGraph(
                url=NEO4J_URI,
                username=NEO4J_USERNAME,
                password=NEO4J_PASSWORD
            )
            
            print("✅ Neo4j connection established")
            return graph
            
        except Exception as e:
            print(f"⚠️ Neo4j connection failed: {e}")
            print("🔄 Falling back to FAISS-only mode")
            return None
    
    def load_data(self):
        """Load information from NWMSU website about the MS-ACS program"""
        print("📥 Loading data from NWMSU website...")
        print("    🌐 Scraping web pages for MS-ACS program information...")
        
        # URLs of NWMSU pages with MS-ACS program information
        urls = [
            "https://www.nwmissouri.edu/csis/msacs/",
            "https://www.nwmissouri.edu/csis/msacs/about.htm",
            "https://www.nwmissouri.edu/academics/graduate/masters/applied-computer-science.htm",
            "https://www.nwmissouri.edu/csis/msacs/apply/index.htm",
            "https://www.nwmissouri.edu/csis/msacs/courses.htm",
            "https://www.nwmissouri.edu/csis/msacs/FAQs.htm",
            "https://www.nwmissouri.edu/csis/msacs/contact.htm"
        ]
        
        # Use LangChain's web loader to get the content
        loader = WebBaseLoader(urls)
        loader.requests_kwargs = {'verify': False}  # Handle SSL issues
        docs = loader.load()
        
        # Split large documents into smaller chunks for better processing
        print("    ✂️ Splitting documents into manageable chunks...")
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,    # Each chunk will be ~800 characters
            chunk_overlap=100  # Overlap to maintain context
        )
        chunks = splitter.split_documents(docs)
        
        print(f"✅ Successfully loaded {len(chunks)} document chunks")
        print(f"📄 Total pages processed: {len(docs)}")
        return chunks
    
    def create_vector_store(self, documents):
        """Create a vector database for similarity search"""
        print("🔍 Creating vector database...")
        print("    🧮 Converting text to numerical vectors...")
        
        # Create FAISS vector store - this will store our document embeddings
        vector_store = FAISS.from_documents(documents, self.embeddings)
        
        print("✅ Vector database created successfully!")
        print("    📊 Your chatbot can now find similar documents quickly")
        return vector_store
    
    def retrieve_context(self, question: str, vector_store, k: int = 3) -> str:
        """Find relevant information for the user's question"""
        print(f"🔍 Searching for relevant information...")
        print(f"    ❓ Question: '{question}'")
        
        # Use similarity search to find the most relevant documents
        docs = vector_store.similarity_search(question, k=k)
        
        # Combine the retrieved documents into context
        context = "\n\n".join([doc.page_content for doc in docs])
        
        print(f"    📄 Found {len(docs)} relevant documents")
        print(f"    📝 Context length: {len(context)} characters")
        return context
    
    def create_qa_chain(self, vector_store):
        """Create the question-answering system"""
        print("⚙️ Setting up question-answering system...")
        
        def retriever_func(question: str) -> str:
            return self.retrieve_context(question, vector_store)
        
        # Template for how the AI should answer questions
        template = """Answer the question based on the context below. Be specific and accurate.

Context: {context}

Question: {question}

Answer concisely:"""
        
        prompt = ChatPromptTemplate.from_template(template)
        
        # Create the complete chain: retrieve context -> generate answer
        chain = (
            RunnableParallel({
                "context": lambda x: retriever_func(x["question"]),
                "question": lambda x: x["question"]
            })
            | prompt
            | self.llm
            | StrOutputParser()
        )
        
        print("✅ Question-answering system ready!")
        return chain
    
    def setup(self):
        """Setup the complete RAG chatbot system"""
        print("🔧 Setting up your RAG chatbot...")
        print("=" * 50)
        
        # Step 1: Load data from NWMSU website
        documents = self.load_data()
        
        # Step 2: Create vector database
        self.vector_store = self.create_vector_store(documents)
        
        # Step 3: Create question-answering system
        self.qa_chain = self.create_qa_chain(self.vector_store)
        
        print("\n🎉 Your RAG chatbot is ready!")
        print("💬 You can now ask questions about the NWMSU MS-ACS program!")
    
    def ask_question(self, question: str) -> str:
        """Ask a question and get an answer"""
        try:
            print(f"\n🤖 Processing your question...")
            answer = self.qa_chain.invoke({"question": question})
            return answer
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    
    def ask_with_context(self, question: str) -> Tuple[str, str]:
        """Ask a question and return both answer and context (for learning purposes)"""
        try:
            # Get context that was used to answer the question
            context = self.retrieve_context(question, self.vector_store)
            
            # Get the answer
            answer = self.qa_chain.invoke({"question": question})
            
            return answer, context
        except Exception as e:
            return f"Error: {str(e)}", f"Context retrieval failed: {str(e)}"

print("✅ Your RAG Chatbot class is ready!")
print("🎓 Students can now build their own chatbot!")


## 🚀 Initialize the RAG System

Let's create and setup our RAG chatbot. This will take a few minutes as it downloads models and processes data.


In [ ]:
# Initialize your RAG chatbot
print("🚀 Creating your RAG chatbot...")
chatbot = NWMSURAGChatbot(use_gpu=False)

# Setup the complete system (this will take a few minutes)
chatbot.setup()

print("\n🎉 Congratulations! Your RAG chatbot is ready!")
print("💬 You can now ask questions about the NWMSU MS-ACS program!")


## 🎯 Step 4: Test Your RAG Chatbot!

Now let's test your chatbot by asking questions about the NWMSU MS-ACS program. You'll see how RAG works by showing the context that was retrieved before generating each answer.


In [ ]:
def demo_rag(question: str):
    """Test your RAG chatbot and see how it works"""
    print(f"\n{'='*60}")
    print(f"❓ QUESTION: {question}")
    print(f"{'='*60}")
    
    # Get answer and context from your chatbot
    answer, context = chatbot.ask_with_context(question)
    
    # Show the context that was retrieved (this is the RAG part!)
    print(f"\n🔍 CONTEXT RETRIEVED BY YOUR CHATBOT:")
    print(f"{'='*60}")
    print(f"{context[:500]}..." if len(context) > 500 else context)
    print(f"\n📊 Context length: {len(context)} characters")
    
    # Show the final answer
    print(f"\n🤖 YOUR CHATBOT'S ANSWER:")
    print(f"{'='*60}")
    print(f"{answer}")
    
    return answer, context

print("✅ Demo function ready!")
print("🎓 Now you can test your RAG chatbot!")


## 🧪 Step 5: Test Your Chatbot with Sample Questions

Let's test your RAG chatbot with some common questions about the MS-ACS program. Watch how it retrieves context before answering!


In [ ]:
# Test Question 1: GPA Requirements
print("🎓 Testing your chatbot with a GPA question...")
demo_rag("What is the minimum GPA required for MS-ACS program?")


In [ ]:
# Test Question 2: TOEFL Requirements
print("\n🎓 Testing your chatbot with a TOEFL question...")
demo_rag("What are the TOEFL score requirements?")


In [ ]:
# Test Question 3: Program Advisor
print("\n🎓 Testing your chatbot with an advisor question...")
demo_rag("Who is the program advisor for MS-ACS?")


In [ ]:
# Test Question 4: Duolingo Requirements
print("\n🎓 Testing your chatbot with a Duolingo question...")
demo_rag("What is the Duolingo score requirement?")


## 🎮 Step 6: Interactive Chatbot Interface

Now you can ask your own questions! Use the interface below to test your RAG chatbot with any questions about the MS-ACS program.


In [ ]:
# Interactive chatbot interface for students
def interactive_rag(question):
    if question.strip():
        return demo_rag(question)
    else:
        print("Please enter a question!")

# Create an interactive interface for your chatbot
print("🎓 Create your own chatbot interface!")
print("💬 Ask any question about the NWMSU MS-ACS program:")

question_widget = widgets.Textarea(
    value='',
    placeholder='Ask a question about MS-ACS program... (e.g., "What courses are offered?")',
    description='Your Question:',
    layout=widgets.Layout(width='100%', height='100px')
)

button = widgets.Button(description="Ask Your Chatbot", button_style='success')

def on_button_click(b):
    if question_widget.value.strip():
        print(f"\n🎓 Student Question: {question_widget.value}")
        demo_rag(question_widget.value)
    else:
        print("Please enter a question!")

button.on_click(on_button_click)

# Display the interactive interface
display(widgets.VBox([
    widgets.HTML("<h3>🤖 Your RAG Chatbot Interface</h3>"),
    question_widget, 
    button
]))


## 🗄️ Neo4j Graph Database Features

Let's explore the advanced features that Neo4j brings to our RAG system:


In [ ]:
# Display graph statistics
def show_graph_stats():
    """Display Neo4j graph statistics"""
    print("📊 Neo4j Graph Statistics:")
    print("=" * 40)
    
    stats = chatbot.get_graph_stats()
    for stat in stats:
        print(f"• {stat['label']}: {stat['count']}")
    
    print("\n🔍 Graph Structure:")
    print("Program → Requirements → Faculty")
    print("   ↓           ↓          ↓")
    print("MS-ACS    GPA, TOEFL   Dr. Ajay Bandi")
    print("          Duolingo, IELTS")

show_graph_stats()


In [ ]:
# Demonstrate structured vs unstructured retrieval
def compare_retrieval_methods(question: str):
    """Compare structured (Neo4j) vs unstructured (vector) retrieval"""
    print(f"\n🔍 RETRIEVAL COMPARISON")
    print(f"{'='*50}")
    print(f"Question: {question}")
    
    # Structured retrieval (Neo4j)
    print(f"\n🗄️ STRUCTURED RETRIEVAL (Neo4j):")
    print(f"{'='*40}")
    structured = chatbot.retrieve_structured_context(question)
    print(f"Result: {structured}")
    
    # Unstructured retrieval (Vector)
    print(f"\n📄 UNSTRUCTURED RETRIEVAL (Vector):")
    print(f"{'='*40}")
    docs = chatbot.vector_store.similarity_search(question, k=2)
    unstructured = "\n\n".join([doc.page_content[:200] + "..." for doc in docs])
    print(f"Result: {unstructured}")
    
    # Combined (RAG)
    print(f"\n🤖 COMBINED RAG RETRIEVAL:")
    print(f"{'='*40}")
    answer, context = chatbot.ask_with_context(question)
    print(f"Answer: {answer}")
    print(f"Context length: {len(context)} characters")

# Test the comparison
compare_retrieval_methods("What are the admission requirements?")


## 🎯 Advanced Neo4j RAG Features

### 🔍 **What Makes Neo4j RAG Special:**

1. **Structured Data**: Neo4j stores entities and relationships
2. **Graph Queries**: Cypher queries for precise data retrieval
3. **Hybrid Retrieval**: Combines structured + unstructured data
4. **Relationship Awareness**: Understands connections between entities
5. **Precise Answers**: More accurate responses with structured context

### 🛠️ **Technical Advantages:**

- **Entity Recognition**: Automatically extracts Program, Requirements, Faculty
- **Relationship Mapping**: Creates connections between entities
- **Structured Queries**: Precise data retrieval with Cypher
- **Hybrid Context**: Combines graph + vector search
- **Better Accuracy**: More precise answers with structured data


In [ ]:
# Advanced Neo4j query demonstration
def demonstrate_neo4j_queries():
    """Demonstrate different types of Neo4j queries"""
    if not chatbot.graph:
        print("⚠️ Neo4j not available for advanced queries")
        return
    
    print("🔍 NEO4J QUERY DEMONSTRATIONS")
    print("=" * 50)
    
    # Query 1: Get all requirements
    print("\n1️⃣ All Requirements:")
    result = chatbot.graph.query("""
        MATCH (p:Program)-[:HAS_REQUIREMENT]->(r:Requirement)
        RETURN r.name, r.value, r.description
    """)
    for row in result:
        print(f"• {row['r.name']}: {row['r.value']} - {row['r.description']}")
    
    # Query 2: Get faculty information
    print("\n2️⃣ Faculty Information:")
    result = chatbot.graph.query("""
        MATCH (f:Faculty)-[:ADVISES]->(p:Program)
        RETURN f.name, f.role, f.email
    """)
    for row in result:
        print(f"• {row['f.name']} ({row['f.role']}): {row['f.email']}")
    
    # Query 3: Get program structure
    print("\n3️⃣ Program Structure:")
    result = chatbot.graph.query("""
        MATCH (p:Program)-[r]->(n)
        RETURN type(r) as relationship, count(n) as count
        ORDER BY count DESC
    """)
    for row in result:
        print(f"• {row['relationship']}: {row['count']} entities")

demonstrate_neo4j_queries()


## 📚 How RAG Works - Educational Explanation

### 🔍 **Retrieval-Augmented Generation (RAG) Process:**

1. **📥 Data Ingestion**: Load documents from NWMSU website
2. **✂️ Text Splitting**: Break documents into manageable chunks
3. **🔤 Embedding Creation**: Convert text chunks to numerical vectors
4. **🗄️ Vector Storage**: Store embeddings in a searchable database
5. **🔍 Context Retrieval**: Find relevant chunks for user questions
6. **🤖 Answer Generation**: Use retrieved context to generate accurate answers

### 🎯 **Why RAG is Important:**

- **🎯 Accuracy**: Provides specific, factual information
- **📚 Knowledge Base**: Can access up-to-date information
- **🔍 Transparency**: Shows what information was used
- **🎓 Educational**: Demonstrates how AI uses context

### 🛠️ **Technical Components:**

- **Embeddings**: `sentence-transformers/all-MiniLM-L6-v2`
- **LLM**: `google/flan-t5-base`
- **Vector Store**: FAISS (Facebook AI Similarity Search)
- **Framework**: LangChain


## ⚖️ RAG vs Traditional AI Comparison

Let's compare how RAG provides better answers than traditional AI:


In [ ]:
def compare_rag_vs_traditional(question: str):
    """Compare RAG vs traditional AI responses"""
    print(f"\n{'='*80}")
    print(f"❓ QUESTION: {question}")
    print(f"{'='*80}")
    
    # RAG Response
    print(f"\n🔍 RAG RESPONSE (with context):")
    print(f"{'='*40}")
    rag_answer, context = chatbot.ask_with_context(question)
    print(f"Answer: {rag_answer}")
    print(f"Context used: {len(context)} characters")
    
    # Traditional AI (without context)
    print(f"\n🤖 TRADITIONAL AI (no context):")
    print(f"{'='*40}")
    traditional_answer = chatbot.llm.invoke(question)
    print(f"Answer: {traditional_answer}")
    print(f"Context used: 0 characters (no retrieval)")
    
    print(f"\n📊 COMPARISON:")
    print(f"- RAG uses {len(context)} characters of specific context")
    print(f"- Traditional AI relies only on training data")
    print(f"- RAG provides more accurate, up-to-date information")

# Example comparison
compare_rag_vs_traditional("What is the minimum GPA required for MS-ACS program?")


## 🎓 Educational Insights

### 🔍 **What Students Learn from This Demo:**

1. **RAG Architecture**: How retrieval and generation work together
2. **Vector Embeddings**: How text is converted to numerical representations
3. **Similarity Search**: How relevant documents are found
4. **Context Injection**: How retrieved context improves AI responses
5. **Transparency**: How to show what information was used

### 🛠️ **Technical Skills Demonstrated:**

- **LangChain**: Modern framework for LLM applications
- **Vector Databases**: FAISS for efficient similarity search
- **Embeddings**: Sentence transformers for semantic understanding
- **Document Processing**: Web scraping and text chunking
- **Interactive UI**: Jupyter widgets for user interaction

### 🎯 **Real-World Applications:**

- **Customer Support**: Company-specific knowledge bases
- **Educational**: Course-specific information systems
- **Research**: Academic paper question-answering
- **Business**: Internal documentation systems


## 🎉 Conclusion

### ✅ **What We Accomplished:**

1. **Built a complete RAG system** from scratch
2. **Demonstrated context retrieval** in real-time
3. **Showed transparency** in AI responses
4. **Created an interactive demo** for students
5. **Explained the technical concepts** behind RAG

### 🚀 **Next Steps for Students:**

1. **Experiment with different questions**
2. **Try different embedding models**
3. **Modify the context retrieval parameters**
4. **Add new data sources**
5. **Implement advanced RAG techniques**

### 📚 **Key Takeaways:**

- **RAG provides better answers** by using relevant context
- **Transparency is important** for AI systems
- **Vector embeddings** enable semantic search
- **Context retrieval** is the key to RAG success
- **Interactive demos** help students understand complex concepts

---

**🎓 This notebook demonstrates how to build and understand RAG systems for educational purposes!**


In [ ]:
# Final interactive demo
print("🎉 RAG Demo Complete!")
print("\nTry asking your own questions about the MS-ACS program:")
print("- What are the admission requirements?")
print("- Who can I contact for more information?")
print("- What courses are available?")
print("- What is the program duration?")
print("\nUse the interactive widget above to ask questions!")
